In [ ]:
mutable struct Pools{FT<:AbstractFloat}
    POMo::FT
    POMh::FT
    MOM ::FT
    DOM ::FT
    QOM ::FT
    MBA ::FT 
    MBD ::FT
    EPO ::FT 
    EPH ::FT 
    EM  ::FT 
    # PTT ::FT # protist 
    # VIR ::FT # virus
end



In [ ]:
struct Flux_POMo end 
struct Flux_POMh end 
struct Flux_MOM end
struct Flux_DOM end
struct Flux_MBA end
struct Flux_MBD end

Base.@kwdef mutable struct Fluxes{FT<:AbstractFloat}
    pomo_dom ::FT = 0.0
    pomo_mom ::FT = 0.0
    pomh_dom ::FT = 0.0
    pomh_mom ::FT = 0.0
    mom_dom ::FT  = 0.0
    dom_mba ::FT  = 0.0
    dom_qom ::FT  = 0.0
    qom_dom ::FT  = 0.0
    co2_maintn_mba ::FT = 0.0
    co2_growth     ::FT = 0.0
    mba_pomh ::FT = 0.0
    mba_pomo ::FT = 0.0
    mba_dom  ::FT = 0.0
    mba_mbd  ::FT = 0.0
    mba_eph  ::FT = 0.0
    mba_epo  ::FT = 0.0
    mba_em   ::FT = 0.0
    mbd_mba  ::FT = 0.0
    co2_maintn_mbd ::FT = 0.0
end

In [6]:
CN_Litter = Vector(undef,3); # 1:litter cn to pomo; 2: litter cn to pomh; 3: litter cn to DOM 

3-element Vector{Any}:
 #undef
 #undef
 #undef

In [ ]:
rCN_LIG2LAB = sINI%dINI(26)

sINI % CN_LITT(2) = const_CN_Cellulose
sINI % CN_WOOD(2) = const_CN_Cellulose
sINI % CN_ROOT(2) = const_CN_Cellulose

! sINI % CN_LITT(1) = frac(1)/(1.D0/sINI%CN_LITT_avg - frac(3)/sINI % CN_LITT(3) - frac(2)/sINI % CN_LITT(2))
frac = sINI%SIN_frac
sINI % CN_LITT(3) = (frac(1)/rCN_LIG2LAB + frac(3))/(1.D0/sINI%CN_LITT_avg - frac(2)/sINI % CN_LITT(2))
sINI % CN_LITT(1) = sINI % CN_LITT(3)*rCN_LIG2LAB

In [ ]:
##4 developing functions: carbon pools 

function NPools!(par::SoilPar,cpools::Pools,cfluxes::Fluxes,input_c,
                 npools::Pools,nfluxes::Fluxes,cnpools::Pools,CN_Litter)
"""
organic nitrogen balance
"""
    @unpack POMo,POMh,MOM,DOM,QOM,MBA,MBD,EPO,EPH,EM = cpools
    # @unpack EPO,EPH,EM = pools
    litter_pomo = input_c.litter_pomo_array/CN_Litter[1] 
    litter_pomh = input_c.litter_pomh_array/CN_Litter[2]
    litter_dom = input_c.litter_dom_array/CN_Litter[3]

    pomo_dec = MM(par,cpools,Flux_POMo)/cnpools.POMo; pomh_dec = MM(par,cpools,Flux_POMh)/cnpools.POMh;
    mom_dec = MM(par,cpools,Flux_MOM)/cnpools.MOM; dom_dec = MM(par,cpools,Flux_DOM)/cnpools.DOM;

    pomo_dom,pomo_mom = Flux!(par,cpools,fluxes,Flux_POMo); 
    pomo_dom = pomo_dom/cnpools.POMo;
    pomo_mom = pomo_mom/cnpools.POMo;
    
    pomh_dom,pomh_mom = Flux!(par,cpools,fluxes,Flux_POMh); 
    dom_mba, dom_qom, qom_dom = Flux!(par,pools,fluxes,Flux_DOM)
    
    mba_mortality, mba_dom, mba_pomo, mba_pomh,
    mba_mbd, mba_CO2_growth, mba_CO2_maintn = 
    Flux!(par,pools,fluxes,Flux_MBA);

    mbd_mba, mbd_CO2_maintn = Flux!(par,pools,fluxes,Flux_MBD)

    epo_dom,eph_dom,em_dom = EnzymeTurnover(par,pools)

    mba_eph, mba_epo, mba_em = EnzymeProduction(par,pools)

    pools.POMo = POMo - pomo_dec + mba_pomo + litter_pomo
    pools.POMh = POMh - pomh_dec + mba_pomh + litter_pomh
    pools.MOM = MOM - mom_dec + pomo_mom + pomh_mom 
    pools.DOM = (DOM - dom_dec - dom_qom + qom_dom + litter_dom + pomh_dom 
                + pomo_dom + mba_dom + epo_dom + eph_dom + em_dom)
    pools.QOM = QOM - qom_dom + dom_qom

    pools.MBA = (MBA - mba_mortality - mba_mbd - mba_CO2_growth 
                - mba_CO2_maintn + dom_mba + mbd_mba
                - mba_eph - mba_epo - mba_em)

    pools.MBD = MBD - mbd_mba - mbd_CO2_maintn + mba_mbd

    pools.EPO = EPO + mba_epo - epo_dom
    pools.EPH = EPH + mba_eph - eph_dom
    pools.EM = EM + mba_em - em_dom
    
    return nothing 
end